In [3]:
import requests as rq
import pandas as pd
from datetime import datetime, timedelta
from IPython import display

In [4]:
class lentaRu_parser:

    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])

        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])

        return search_table


    def get_articles(self,
                     param_dict,
                     time_step = 10,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print('Parsing articles from '\
                  + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = pd.concat([out, self._get_search_table(param_copy)], axis=0, ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_excel("lenta_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')

        return out

* __query__ - поисковой запрос (ключевое слово)

* __offset__ - cмещение поисковой выдачи (от 0 до __size__)

* __size__ - количество статей. Ограничено время запроса, точного лимита нет. Условно, до 1000

* __sort__ - сортировка по дате: (2) - по убыванию, (3) - по возрастанию; по релевантности (1)

* __title_only__ - точная фраза в заголовке (1)

* __domain__ - ?

* __material__ - материалы: Все материалы (0). Новость (1). ["0", "1", "2", "3", "4", ...]

* __block__ - рубрика: Экономика (4). Все рубрики (0). ["0", "1", "2", "3", "4", ...]

* __dateFrom__ - с даты

* __dateTo__ - по дату

_Чтобы не специфировать параметр, оставляем поле пустым_

In [7]:
use_parser = "LentaRu"

query = ''# Например: Акции Сбера
offset = 0
size = 500
sort = "1"
title_only = "0"
domain = "1"
material = "1"
bloc = "4"# экономика
dateFrom = '2024-06-01'
dateTo = "2024-12-08"

if use_parser == "LentaRu":
    param_dict = {'query'     : query,
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material,
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': 'Акции Сбера', 'from': '0', 'size': '500', 'dateFrom': '2024-06-01', 'dateTo': '2024-12-08', 'sort': '1', 'title_only': '0', 'type': '1', 'bloc': '4', 'domain': '1'}


In [16]:
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 60,
                         save_every = 5,)

Parsing articles from 2024-06-01 to 2024-07-31
Parsing articles from 2024-08-01 to 2024-09-30
Parsing articles from 2024-10-01 to 2024-11-30
Parsing articles from 2024-12-01 to 2024-12-08
Finish


In [20]:
tbl.head()

,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,1656597,https://lenta.ru/news/2024/07/11/gep/,Аналитики высказали предположение о быстром за...,1720702332,1720702504,1,1,0,0,4,[11],https://icdn.lenta.ru/images/2024/07/11/15/202...,1720702332,Фото: Scott Graham / Unsplash Григорий Шугаев ...,Аналитики высказали предположение о быстром за...,"По мнению аналитиков «БКС Экспресс», на закрыт..."
1,1656749,https://lenta.ru/news/2024/07/11/privlekatelnost/,Аналитики зафиксировали привлекательность акци...,1720716240,1720716379,1,1,0,0,4,[11],https://icdn.lenta.ru/images/2024/07/11/19/202...,1720716240,Фото: muse studio / Shutterstock / Fotodom Гр...,Аналитики зафиксировали привлекательность акци...,11 июля 2024 года состоялась дивидендная отсеч...
2,1656526,https://lenta.ru/news/2024/07/11/min/,Российский фондовый рынок упал до минимума за год,1720699320,1720699927,1,1,0,1,4,[8],https://icdn.lenta.ru/images/2024/07/11/14/202...,1720699320,Фото: Екатерина Якель / «Лента.ру» Платон Щуки...,Российский фондовый рынок упал до минимума за год,Отдельно на индекс влияет технический ...— так...
3,1654997,https://lenta.ru/news/2024/07/09/rost/,Сбер зафиксировал рост прибыли за полугодие,1720510850,1720511414,1,1,0,0,4,[11],https://icdn.lenta.ru/images/2024/07/09/10/202...,1720510850,Фото: Григорий Сысоев / РИА Новости Григорий Ш...,Сбер зафиксировал рост прибыли за полугодие,"По его словам, <b>акции</b> <b>Сбера</b> по-пр..."
4,1659847,https://lenta.ru/news/2024/07/17/vazhnost/,Аналитики отметили важность выплаты Сбером див...,1721233705,1721233726,1,1,0,0,4,"[11, 145]",https://icdn.lenta.ru/images/2024/07/17/19/202...,1721233705,Фото: Александр Сухов / РИА Новости Григорий Ш...,Аналитики отметили важность выплаты Сбером див...,Фото: Александр Сухов / РИА Новости Григорий Ш...
